In [1]:
#Loading the libraries
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import numpy
import PIL
import pandas as pd
from PIL import Image

/Users/pratham/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from PIL import Image

In [3]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten

In [4]:
tf.keras.__version__

'2.0.8-tf'

In [5]:
#Working Directory
import os
#For Train Images
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD/Train")

In [6]:
#for train data
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD")
train = pd.read_csv("Train.csv")

In [7]:
np.shape(train)

(19906, 2)

In [8]:
train.head(n=5)

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [9]:
#For test data
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/test_Bh8pGW3")
test = pd.read_csv("Test.csv")

In [10]:
np.shape(test)

(6636, 1)

In [11]:
train_path="/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD/Train/"
test_path="/Users/pratham/Desktop/ML:DL Practice/Age Detection/test_Bh8pGW3/Test/"

In [12]:
#load all the train images in a list
relevant_path = "/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD/Train"
included_extensions = ['jpg', 'bmp', 'png', 'gif']
file_names = [fn for fn in os.listdir(relevant_path)
              if any(fn.endswith(ext) for ext in included_extensions)]

In [13]:
from keras.preprocessing import image

Using TensorFlow backend.


In [14]:
from keras.applications.vgg16 import preprocess_input


In [15]:
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/train_DETg9GD/Train")

In [16]:
train_img=[]
for i in range(len(file_names)):

    temp_img=image.load_img(file_names[i],target_size=(224,224))

    temp_img=image.img_to_array(temp_img)

    train_img.append(temp_img)

#converting train images to array and applying mean subtraction processing

train_img=np.array(train_img) 
train_img=preprocess_input(train_img)

In [17]:
import scipy.ndimage

In [18]:
os.chdir("/Users/pratham/Desktop/ML:DL Practice/Age Detection/test_Bh8pGW3/Test")

In [19]:
#load all the test images in a list
relevant_path = "/Users/pratham/Desktop/ML:DL Practice/Age Detection/test_Bh8pGW3/Test"
included_extensions = ['jpg', 'bmp', 'png', 'gif']
file_names = [fn for fn in os.listdir(relevant_path)
              if any(fn.endswith(ext) for ext in included_extensions)]

In [20]:
test_img=[]
for i in range(len(file_names)):

    temp_img=image.load_img(file_names[i],target_size=(224,224))

    temp_img=image.img_to_array(temp_img)

    test_img.append(temp_img)

test_img=np.array(test_img) 
test_img=preprocess_input(test_img)

In [21]:
from keras.models import Model

In [22]:
def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    model = VGG16(weights='imagenet', include_top=True)

    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-1].outbound_nodes = []

    x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)

#To set the first 8 layers to non-trainable (weights will not be updated)

    for layer in model.layers[:8]:

       layer.trainable = False

# Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [23]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss

In [24]:
train_y=np.asarray(train['Class'])

le = LabelEncoder()

train_y = le.fit_transform(train_y)

train_y=to_categorical(train_y)

train_y=np.array(train_y)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.2, random_state=42)

In [26]:
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 3 
batch_size = 32 
nb_epoch = 1

In [27]:
# Load our model
model = vgg16_model(img_rows, img_cols, channel, num_classes)

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [2]:
# Start Fine-tuning
model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

In [30]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

3982/3982 [==============================] - 4598s 1s/step


In [1]:
#Final predictions
predictions_test = model.predict(test_img,batch_size=batch_size, verbose=1)

In [32]:
# Cross-entropy loss score
score = log_loss(Y_valid, predictions_valid)

In [33]:
print(score)

1.103295666527233
